Issues to solve
* Far too many duplicates being collected in the dataset
* Handle mispelled and redundant words
* ~~removing mentions and hashtag~~
* ~~removing links, special characters, punctuation marks~~
* ~~stopwords removal~~
* ~~contractions handling~~
* ~~stemming and lemmatization~~ did only lemmatization

In [40]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import re
import fnmatch
import string
from urllib.parse import urlparse
import contractions

import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('words')
from nltk.tokenize import sent_tokenize, word_tokenize, TweetTokenizer
from nltk.corpus import words, stopwords
from nltk.metrics.distance import jaccard_distance, edit_distance
from nltk.util import ngrams
from nltk.stem import WordNetLemmatizer


[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/nitanshjain/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/nitanshjain/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     /Users/nitanshjain/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [41]:
tweets_df = pd.read_csv('/Users/nitanshjain/Documents/Thapar 4th Sem/Machine Learing/Machine_Learning_Project/data/20221105_193249_tweets.csv')
tweets_df.head()

,user_id,tweet_id,username,location,following,followers,twt_created_at,total_tweets,retweet_count,text,hashtags,mentions
0,4.757313e+08,1.588852e+18,Rameshbirla1,Thane,1385,261,2022-11-05 11:14:41+00:00,6998,0,@cachiraggala @IncomeTaxIndia @FinMinIndia @In...,[],"[{'screen_name': 'cachiraggala', 'name': 'CA C..."
1,1.556620e+18,1.588852e+18,LetareLugun,NaN,85,183,2022-11-05 11:13:22+00:00,21,0,@BrijSharma1980 @therahulads @CryptoAdsIndia @...,"[{'text': 'Crypto', 'indices': [149, 156]}, {'...","[{'screen_name': 'BrijSharma1980', 'name': 'Br..."
2,2.402726e+09,1.588852e+18,shashikapoor010,karkardooma court,226,61,2022-11-05 11:13:16+00:00,285,0,@esichq If the hospital where the patient is a...,[],"[{'screen_name': 'esichq', 'name': 'ESIC - स्व..."
3,1.420950e+18,1.588852e+18,ChinnasamyPitc2,"Chennai, India",143,27,2022-11-05 11:12:46+00:00,3599,0,@purnach85329713 @PrCCA_WZ_DOT @DoT_India @dev...,[],"[{'screen_name': 'purnach85329713', 'name': 'p..."
4,1.580600e+18,1.588852e+18,shahrekhass,"Srinagar, Jammu & Kashmir",45,8,2022-11-05 11:11:58+00:00,67,0,@OfficeOfLGJandK \nNegligence by IGP traffic S...,[],"[{'screen_name': 'OfficeOfLGJandK', 'name': 'O..."


In [42]:
tweets_df.shape

(2793, 12)

In [43]:
print('Shape of dataset before removal of duplicates is {}'.format(tweets_df.shape))
tweets_df.drop_duplicates(subset=['tweet_id'], inplace=True)
print('Shape of dataset after removal of duplicates is {}'.format(tweets_df.shape))

Shape of dataset before removal of duplicates is (2793, 12)
Shape of dataset after removal of duplicates is (492, 12)


In [44]:
tweets_df

,user_id,tweet_id,username,location,following,followers,twt_created_at,total_tweets,retweet_count,text,hashtags,mentions
0,4.757313e+08,1.588852e+18,Rameshbirla1,Thane,1385,261,2022-11-05 11:14:41+00:00,6998,0,@cachiraggala @IncomeTaxIndia @FinMinIndia @In...,[],"[{'screen_name': 'cachiraggala', 'name': 'CA C..."
1,1.556620e+18,1.588852e+18,LetareLugun,NaN,85,183,2022-11-05 11:13:22+00:00,21,0,@BrijSharma1980 @therahulads @CryptoAdsIndia @...,"[{'text': 'Crypto', 'indices': [149, 156]}, {'...","[{'screen_name': 'BrijSharma1980', 'name': 'Br..."
2,2.402726e+09,1.588852e+18,shashikapoor010,karkardooma court,226,61,2022-11-05 11:13:16+00:00,285,0,@esichq If the hospital where the patient is a...,[],"[{'screen_name': 'esichq', 'name': 'ESIC - स्व..."
3,1.420950e+18,1.588852e+18,ChinnasamyPitc2,"Chennai, India",143,27,2022-11-05 11:12:46+00:00,3599,0,@purnach85329713 @PrCCA_WZ_DOT @DoT_India @dev...,[],"[{'screen_name': 'purnach85329713', 'name': 'p..."
4,1.580600e+18,1.588852e+18,shahrekhass,"Srinagar, Jammu & Kashmir",45,8,2022-11-05 11:11:58+00:00,67,0,@OfficeOfLGJandK \nNegligence by IGP traffic S...,[],"[{'screen_name': 'OfficeOfLGJandK', 'name': 'O..."
...,...,...,...,...,...,...,...,...,...,...,...,...
1713,7.579238e+07,1.582220e+18,bejonmisra,Hyderabad 500081 India,392,998,2022-10-18 04:07:41+00:00,1998,1,https://t.co/cIVZGkZaMU Hon’ble PM @narendramo...,[],"[{'screen_name': 'narendramodi', 'name': 'Nare..."
1714,1.150000e+18,1.582210e+18,icccedu,"Bengaluru, India",63,28,2022-10-18 03:23:42+00:00,1417,1,IN GOVT MEDICAL #DMCDARBHANGA UR CATEGORY OPTH...,"[{'text': 'DMCDARBHANGA', 'indices': [16, 29]}...","[{'screen_name': 'NitishKumar', 'name': 'Nitis..."
1715,1.420000e+18,1.582120e+18,DrRishiRajSinh1,Bhopal Madhya Pradesh,217,81,2022-10-17 21:14:15+00:00,119,1,#IndianMedicalService should be there in best ...,"[{'text': 'IndianMedicalService', 'indices': [...","[{'screen_name': 'FAIMA_INDIA_', 'name': 'FAIM..."
1716,4.791493e+07,1.582070e+18,sunilbhatt5,Ahmedabad,151,109,2022-10-17 17:54:56+00:00,8744,0,@smcgujarat @nirliptrai @PMOIndia @HMOIndia @A...,[],"[{'screen_name': 'smcgujarat', 'name': 'SMC Gu..."


In [45]:
tweets_df.dtypes

user_id           float64
tweet_id          float64
username           object
location           object
following           int64
followers           int64
twt_created_at     object
total_tweets        int64
retweet_count       int64
text               object
hashtags           object
mentions           object
dtype: object

In [46]:
def data_preprocessing(df):
    """
    One function to rule them all, 
    one function to find them, 
    One function to bring them all, 
    and in the darkness bind them; 
    """
    print('Shape of dataset before removal of tweets with less than 5 words is {}'.format(df.shape))
    
    for tweets in df.loc[:,'text']:
        # count+=1
        # print(tweets)
        tokenizer = TweetTokenizer()
        tweet_id = df.loc[df['text'] == tweets, 'tweet_id'].values[0]
        # print(tweet_id)
        
        # removing links
        list_words = tokenizer.tokenize(tweets)
        x = [word for word in list_words if not urlparse(word).scheme]
        tweets = ' '.join(x)

        # contractions handling
        list_words = tokenizer.tokenize(tweets)
        new_list_words = []
        for word in list_words:
            new_list_words.append(contractions.fix(word))
        list_words = new_list_words
        del(new_list_words)
        tweets = ' '.join(list_words)
        
        # adding space between words and punctuations
        tweets = tweets.replace(',', ' ,').replace('.', ' .').replace('?', ' ?').replace('!', ' !')
        
        # removing hashtags and mentions
        list_words = tokenizer.tokenize(tweets)
        list_words = [word for word in list_words if word[0] not in ('#', '@')]
        list_words = [word for word in list_words if word[0] not in ('▪')]
        tweets = ' '.join(list_words)
        
        # removing punctuations
        tweets = tweets.translate(str.maketrans('', '', string.punctuation))
        
        #removing emojis
        tweets = re.sub(r'[^\x00-\x7F]+', ' ', tweets)
        
        #lower case
        tweets = tweets.lower()
        
        #remove numbers
        tweets = re.sub(r'\d+', '', tweets)
        tweets = re.sub(' +', ' ', tweets)
        
        #removing stopwords
        list_words = tokenizer.tokenize(tweets)
        filtered_words = [word for word in list_words if word not in stopwords.words('english')]
        tweets = ' '.join(filtered_words)
        del(filtered_words)
        
        #lemmatization
        lem = WordNetLemmatizer()
        list_words = tokenizer.tokenize(tweets)
        for word in list_words:
            list_words = list(map(lambda x: x.replace(word, lem.lemmatize(word)), list_words))
        tweets = ' '.join(list_words)
        
        #removing individual letters
        list_words = tokenizer.tokenize(tweets)
        filtered_words = [word for word in list_words if len(word)>2]
        tweets = ' '.join(filtered_words)
        del(filtered_words)
        
        # updating tweets in dataframe
        df.loc[df['tweet_id']==tweet_id, 'text'] = tweets
        
        #remove small tweets
        list_words = tokenizer.tokenize(tweets)
        if len(list_words) <= 5:
            ind_num = df[df['tweet_id']==tweet_id].index
            df.drop(ind_num, inplace=True)
        # break
    print('Shape of dataset after removal of tweets with less than 5 words is {}'.format(df.shape))
    
    return df
        
        
        

In [47]:
tweets_df = data_preprocessing(tweets_df)

Shape of dataset before removal of tweets with less than 5 words is (492, 12)
Shape of dataset after removal of tweets with less than 5 words is (348, 12)
